## Inequalities in Health in England

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests

### Data Gathering

In [4]:
url_local_health="https://wpieconomics.us13.list-manage.com/track/click?u=5331abc1c0dacc833dd4e807b&id=645de758bb&e=9f29c59835"
response=requests.get(url_local_health)
with open("local_health_in_England.csv",mode='wb') as file:
    file.write(response.content)

In [6]:
df=pd.read_csv('local_health_in_England.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6793 entries, 0 to 6792
Data columns (total 4 columns):
 #   Column                                                        Non-Null Count  Dtype 
---  ------                                                        --------------  ----- 
 0   Local Health - Office for Health Improvement and Disparities  6793 non-null   object
 1                                                                 6793 non-null   object
 2    .1                                                           6737 non-null   object
 3    .2                                                           6774 non-null   object
dtypes: object(4)
memory usage: 212.4+ KB


## Data Acessment

In [9]:
df.head()

,Local Health - Office for Health Improvement and Disparities,,.1,.2
0,Geographical references: MSOA 2011 (Middle lev...,,,
1,Code,Label,Life expectancy at birth for males,Life expectancy at birth for females
2,E02000001,City of London 001,91.1,90.2
3,E02000002,Barking and Dagenham 001,79.5,82.5
4,E02000003,Barking and Dagenham 002,78.1,84.2


In [11]:
df.columns

Index(['Local Health - Office for Health Improvement and Disparities', ' ',
       ' .1', ' .2'],
      dtype='object')

#### Outcome of Assessing the Data
- Remove the title
- rename columns
- Treat NaN values
- Strip the city code of the city label
- remove NaN for life expectancy at birth- that's the independent variable

## Data Cleaning

In [10]:
df.columns

Index(['Local Health - Office for Health Improvement and Disparities', ' ',
       ' .1', ' .2'],
      dtype='object')